In [ ]:
# pip install imodels

In [1]:
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, mean_absolute_error, accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error
from datetime import datetime
from datetime import timedelta
from imodels.importance import RandomForestPlusRegressor, RandomForestPlusClassifier, \
    RidgeRegressorPPM, LassoRegressorPPM, IdentityTransformer
from imodels.importance.rf_plus import _fast_r2_score

In [2]:
import multiprocessing
multiprocessing.cpu_count()

12

In [3]:
data = pd.read_csv('PRSA_data_2010.1.1-2014.12.31_1.csv', header=0)

In [4]:
data = data.dropna()

In [5]:

def neg_mae(y_true, y_pred, **kwargs):
    """
    Evaluates negative mean absolute error
    """
    return -mean_absolute_error(y_true, y_pred, **kwargs)

In [13]:
rf_regressor = RandomForestRegressor(n_estimators=50,
                                     min_samples_leaf=3, random_state=331,n_jobs=-1)
rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor)

In [276]:
paper_rmse_2010 = np.array([10.07,34.86,13.58,4.55,10.86,11,16,9.9,28.44,21.45,41,23.83])
paper_rmse_2011 = np.array([8.26,24.5,8.02,11.17,6.94,20.15,16.99,21.48,26.36,20.37,25.35,11.85])
paper_rmse_2012 = np.array([8.44,13.22,14.41,11.36,20.13,15.48,16.84,21.19,11.7,30.96,30.38,5.81])
paper_rmse_2013 = np.array([36.74,25.44,16.26,6.24,13.75,16.37,11.89,9.11,21.38,20.41,19.41,22.72])
paper_rmse_2014 = np.array([26.34,14.83,9.44,13.67,10.44,9.64,13.76,14.31,10.74,28.11,23.78,23.18])

In [6]:
data
data = data.dropna()

In [6]:
# wind_direction_mapping = {
#     'NW': 1,
#     'NE': 2,
#     'SE': 3,
#     'cv': 4
# }

# # Replace the string categories with the numeric values
# data['cbwd'] = data['cbwd'].replace(wind_direction_mapping)

In [7]:
data = pd.get_dummies(data)

In [8]:
data

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_cv
24,25,2010,1,2,0,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0
25,26,2010,1,2,1,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0
26,27,2010,1,2,2,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0
27,28,2010,1,2,3,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0
28,29,2010,1,2,4,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,43820,2014,12,31,19,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0
43820,43821,2014,12,31,20,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0
43821,43822,2014,12,31,21,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0
43822,43823,2014,12,31,22,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0


In [9]:
train_data_by_month, test_data_by_month = {}, {}
train_data_by_year, test_data_by_year = {}, {}

# Splitting data for each month (month-year combination) separately
for year in data['year'].unique():
    for month in data['month'].unique():
        month_year_data = data[(data['year'] == year) & (data['month'] == month)]
        train_data_by_month[(year, month)], test_data_by_month[(year, month)] = train_test_split(month_year_data,
                                                                                                 test_size=0.2, random_state=12345)

# Combine the monthly splits into yearly splits
for year in data['year'].unique():
    yearly_train_data = pd.concat([train_data_by_month[(year, month)] for month in data['month'].unique()])
    yearly_test_data = pd.concat([test_data_by_month[(year, month)] for month in data['month'].unique()])
    train_data_by_year[year] = yearly_train_data
    test_data_by_year[year] = yearly_test_data

# Combine all monthly splits into a single dataset for full range modeling
train_data_full = pd.concat(train_data_by_month.values())
test_data_full = pd.concat(test_data_by_month.values())

In [10]:
train_data_by_year.items()

dict_items([(2010,         No  year  month  day  hour  pm2.5  DEWP  TEMP    PRES     Iws  Is  Ir  \
620    621  2010      1   26    20  274.0   -11  -5.0  1020.0    0.89   0   0   
367    368  2010      1   16     7  114.0   -15 -11.0  1034.0    4.47   0   0   
627    628  2010      1   27     3  213.0   -11  -4.0  1016.0   10.29   0   0   
90      91  2010      1    4    18   28.0   -23 -11.0  1032.0  165.37   0   0   
178    179  2010      1    8    10  135.0   -17 -10.0  1030.0    0.89   0   0   
...    ...   ...    ...  ...   ...    ...   ...   ...     ...     ...  ..  ..   
8436  8437  2010     12   18    12  206.0   -13   3.0  1012.0    1.79   0   0   
8145  8146  2010     12    6     9   15.0   -18   0.0  1035.0  193.97   0   0   
8301  8302  2010     12   12    21  140.0    -7   0.0  1023.0    2.23   0   0   
8501  8502  2010     12   21     5  367.0   -10  -5.0  1013.0    1.79   0   0   
8498  8499  2010     12   21     2  391.0   -10  -4.0  1014.0    3.57   0   0   

      cb

In [11]:
test_data_by_month.keys()

dict_keys([(2010, 1), (2010, 2), (2010, 3), (2010, 4), (2010, 5), (2010, 6), (2010, 7), (2010, 8), (2010, 9), (2010, 10), (2010, 11), (2010, 12), (2011, 1), (2011, 2), (2011, 3), (2011, 4), (2011, 5), (2011, 6), (2011, 7), (2011, 8), (2011, 9), (2011, 10), (2011, 11), (2011, 12), (2012, 1), (2012, 2), (2012, 3), (2012, 4), (2012, 5), (2012, 6), (2012, 7), (2012, 8), (2012, 9), (2012, 10), (2012, 11), (2012, 12), (2013, 1), (2013, 2), (2013, 3), (2013, 4), (2013, 5), (2013, 6), (2013, 7), (2013, 8), (2013, 9), (2013, 10), (2013, 11), (2013, 12), (2014, 1), (2014, 2), (2014, 3), (2014, 4), (2014, 5), (2014, 6), (2014, 7), (2014, 8), (2014, 9), (2014, 10), (2014, 11), (2014, 12)])

In [12]:
rf_regressor = RandomForestRegressor(n_estimators=50,min_samples_leaf=3,
                                     random_state=331,n_jobs=-1)
rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor)


In [12]:
training_results = {}
testing_results = {}
prediction_vectors = pd.DataFrame()
col_drop = ['pm2.5', 'No','year', 'month','day','hour']
for (year, month), train_data in train_data_by_month.items():
    test_data = test_data_by_month[(year, month)]


    X_train = train_data.drop(col_drop, axis=1)
    y_train = train_data['pm2.5'].to_numpy()
    X_test = test_data.drop(col_drop, axis=1)
    y_test = test_data['pm2.5'].to_numpy()
    
    # Initialize RF+
    rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor)
    rf_plus_model.fit(X_train, y_train)
    
    # predictions
    train_pred = rf_plus_model.predict(X_train)
    test_pred = rf_plus_model.predict(X_test)

    # Calculate RMSE
    train_rmse = mean_squared_error(y_train, train_pred, squared=False)
    test_rmse = mean_squared_error(y_test, test_pred, squared=False)
    training_results[(year, month)] = train_rmse
    testing_results[(year, month)] = test_rmse

    # store in dataframe
    train_df = pd.DataFrame({'Year': year, 'Month': month, 'Type': 'Train', 'Y_True': y_train, 'Y_Pred': train_pred})
    test_df = pd.DataFrame({'Year': year, 'Month': month, 'Type': 'Test', 'Y_True': y_test, 'Y_Pred': test_pred})

    # Concatenate with the main DataFrame
    prediction_vectors = pd.concat([prediction_vectors, train_df, test_df])

    # Print results for each month
    print(f"Year: {year}, Month: {month}, Training RMSE: {train_rmse}, Testing RMSE: {test_rmse}")

# Results include: training_results, testing_results, and prediction_vectors

Year: 2010, Month: 1, Training RMSE: 28.06219855087765, Testing RMSE: 38.5537193022401
Year: 2010, Month: 2, Training RMSE: 49.90216708371103, Testing RMSE: 45.96970711472546
Year: 2010, Month: 3, Training RMSE: 41.876150308165116, Testing RMSE: 35.11032099853313
Year: 2010, Month: 4, Training RMSE: 23.52082546717038, Testing RMSE: 36.57962792668829
Year: 2010, Month: 5, Training RMSE: 26.138660662411638, Testing RMSE: 43.69078244426372
Year: 2010, Month: 6, Training RMSE: 26.07220946324403, Testing RMSE: 34.43549480064791
Year: 2010, Month: 7, Training RMSE: 27.834940438810776, Testing RMSE: 42.57616623000923
Year: 2010, Month: 8, Training RMSE: 26.142778186787424, Testing RMSE: 37.95325033359249
Year: 2010, Month: 9, Training RMSE: 46.74668741672338, Testing RMSE: 60.63116773832421
Year: 2010, Month: 10, Training RMSE: 36.08362265185624, Testing RMSE: 49.53423750705735
Year: 2010, Month: 11, Training RMSE: 63.189952267774935, Testing RMSE: 75.28540373196424
Year: 2010, Month: 12, Tra

In [13]:
prediction_vectors

,Year,Month,Type,Y_True,Y_Pred
0,2010,1,Train,274.0,250.523225
1,2010,1,Train,114.0,132.196807
2,2010,1,Train,213.0,177.300678
3,2010,1,Train,28.0,26.619246
4,2010,1,Train,135.0,121.686668
...,...,...,...,...,...
139,2014,12,Test,11.0,22.987379
140,2014,12,Test,9.0,8.285277
141,2014,12,Test,119.0,98.909549
142,2014,12,Test,71.0,69.219733


In [14]:
grouped = prediction_vectors.groupby(['Year', 'Type'])

# Function to calculate RMSE
def calculate_rmse(group):
    return np.sqrt(mean_squared_error(group['Y_True'], group['Y_Pred']))

# Calculate RMSE for each group
yearly_rmse = grouped.apply(calculate_rmse).unstack()
print('annual error by monthly model:')
print(yearly_rmse)

annual error by monthly model:
Type       Test      Train
Year                      
2010  46.490575  37.819589
2011  48.437741  34.064595
2012  43.135437  33.119752
2013  46.658585  37.343240
2014  51.079089  36.839085


In [15]:
training_results.values()

dict_values([28.06219855087765, 49.90216708371103, 41.876150308165116, 23.52082546717038, 26.138660662411638, 26.07220946324403, 27.834940438810776, 26.142778186787424, 46.74668741672338, 36.08362265185624, 63.189952267774935, 39.83690227974915, 18.636261571446365, 52.084098730431954, 19.078167084319833, 32.55371720922852, 19.360504581297395, 32.07809719479406, 31.351720328729897, 30.04331664135062, 43.91636977657398, 37.05155034114965, 36.6131250206199, 37.34731173990122, 56.77729084156379, 25.93228977280587, 26.4447920732958, 24.45386208578873, 26.546622225304503, 33.887343895231226, 27.398630426386116, 34.45903046742415, 17.474897117302643, 41.691658868826536, 36.995910243955045, 28.84811081443199, 68.10412775713843, 48.27418637581725, 37.1238889709738, 20.27888948452465, 24.22685409268405, 34.93282390345045, 20.51080613611921, 14.829010967607164, 32.87733493399166, 48.37697645051142, 33.069262324628774, 31.887876243648662, 56.29186683048487, 49.66562212845513, 40.04212081725237, 34

In [16]:
train_data_by_year.items()

dict_items([(2010,         No  year  month  day  hour  pm2.5  DEWP  TEMP    PRES  cbwd     Iws  \
620    621  2010      1   26    20  274.0   -11  -5.0  1020.0     2    0.89   
367    368  2010      1   16     7  114.0   -15 -11.0  1034.0     1    4.47   
627    628  2010      1   27     3  213.0   -11  -4.0  1016.0     1   10.29   
90      91  2010      1    4    18   28.0   -23 -11.0  1032.0     1  165.37   
178    179  2010      1    8    10  135.0   -17 -10.0  1030.0     2    0.89   
...    ...   ...    ...  ...   ...    ...   ...   ...     ...   ...     ...   
8436  8437  2010     12   18    12  206.0   -13   3.0  1012.0     1    1.79   
8145  8146  2010     12    6     9   15.0   -18   0.0  1035.0     1  193.97   
8301  8302  2010     12   12    21  140.0    -7   0.0  1023.0     4    2.23   
8501  8502  2010     12   21     5  367.0   -10  -5.0  1013.0     1    1.79   
8498  8499  2010     12   21     2  391.0   -10  -4.0  1014.0     2    3.57   

      Is  Ir  
620    0   0  
36

In [17]:
year_training_results = {}
year_testing_results = {}
year_prediction_vectors = pd.DataFrame()
col_drop =['pm2.5', 'No','year', 'month','day','hour']


for year, train_data in train_data_by_year.items():
    test_data = test_data_by_year[year]
    X_train = train_data.drop(col_drop, axis=1)
    y_train = train_data['pm2.5'].to_numpy()
    X_test = test_data.drop(col_drop, axis=1)
    y_test = test_data['pm2.5'].to_numpy()
    
    # Initialize RF+
    rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor)
    rf_plus_model.fit(X_train, y_train)
    
    # Make predictions and calculate metrics
    train_pred = rf_plus_model.predict(X_train)
    test_pred = rf_plus_model.predict(X_test)
    train_rmse = mean_squared_error(y_train, train_pred, squared=False)
    test_rmse = mean_squared_error(y_test, test_pred, squared=False)
    
    # Store and print results
    year_training_results[year] = train_rmse
    year_testing_results[year] = test_rmse
    train_df = pd.DataFrame({'Year': year, 'Type': 'Train', 'Y_True': y_train, 'Y_Pred': train_pred})
    test_df = pd.DataFrame({'Year': year, 'Type': 'Test', 'Y_True': y_test, 'Y_Pred': test_pred})
    year_prediction_vectors = pd.concat([year_prediction_vectors, train_df, test_df])
    print(f"Year: {year}, Training RMSE: {train_rmse}, Testing RMSE: {test_rmse}")


Year: 2010, Training RMSE: 49.20598696618397, Testing RMSE: 58.525477329745
Year: 2011, Training RMSE: 44.691368679383466, Testing RMSE: 58.95753719452501
Year: 2012, Training RMSE: 45.382487163810985, Testing RMSE: 56.10015142212281
Year: 2013, Training RMSE: 50.59292006391516, Testing RMSE: 64.23418292275505
Year: 2014, Training RMSE: 49.14325094620099, Testing RMSE: 66.55011019050096


In [18]:
year_prediction_vectors

,Year,Type,Y_True,Y_Pred
0,2010,Train,274.0,201.337160
1,2010,Train,114.0,140.455175
2,2010,Train,213.0,164.383395
3,2010,Train,28.0,25.224678
4,2010,Train,135.0,98.105788
...,...,...,...,...
1733,2014,Test,11.0,40.555918
1734,2014,Test,9.0,31.222155
1735,2014,Test,119.0,88.762226
1736,2014,Test,71.0,79.600697


In [13]:
train_data_full

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_cv
620,621,2010,1,26,20,274.0,-11,-5.0,1020.0,0.89,0,0,1,0,0,0
367,368,2010,1,16,7,114.0,-15,-11.0,1034.0,4.47,0,0,0,1,0,0
627,628,2010,1,27,3,213.0,-11,-4.0,1016.0,10.29,0,0,0,1,0,0
90,91,2010,1,4,18,28.0,-23,-11.0,1032.0,165.37,0,0,0,1,0,0
178,179,2010,1,8,10,135.0,-17,-10.0,1030.0,0.89,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43519,43520,2014,12,19,7,66.0,-12,0.0,1023.0,22.35,0,0,0,1,0,0
43211,43212,2014,12,6,11,30.0,-20,2.0,1028.0,9.83,0,0,1,0,0,0
43384,43385,2014,12,13,16,44.0,-19,4.0,1026.0,8.04,0,0,0,0,1,0
43593,43594,2014,12,22,9,145.0,-13,-4.0,1026.0,5.37,0,0,1,0,0,0


In [14]:
test_data_full

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_cv
264,265,2010,1,12,0,21.0,-22,-15.0,1031.0,1.78,0,0,0,0,0,1
487,488,2010,1,21,7,18.0,-25,-9.0,1038.0,138.10,0,0,0,1,0,0
528,529,2010,1,23,0,49.0,-17,-4.0,1033.0,172.08,0,0,0,1,0,0
438,439,2010,1,19,6,330.0,-4,-1.0,1021.0,12.07,0,0,0,1,0,0
685,686,2010,1,29,13,12.0,-24,5.0,1022.0,50.07,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43193,43194,2014,12,5,17,11.0,-22,0.0,1025.0,1.79,0,0,1,0,0,0
43189,43190,2014,12,5,13,9.0,-23,3.0,1026.0,36.20,0,0,0,1,0,0
43221,43222,2014,12,6,21,119.0,-14,0.0,1026.0,0.89,0,0,0,0,0,1
43216,43217,2014,12,6,16,71.0,-18,2.0,1025.0,5.37,0,0,0,0,1,0


In [15]:
start = datetime.now()
col_drop = ['pm2.5', 'No','year', 'month','day','hour']
train_data= train_data_full
test_data = test_data_full
X_train = train_data.drop(col_drop, axis=1)
y_train = train_data['pm2.5'].to_numpy()
X_test = test_data.drop(col_drop, axis=1)
y_test = test_data['pm2.5'].to_numpy()

# Initialize RF+
rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor)
rf_plus_model.fit(X_train, y_train)




end = datetime.now()
end-start

datetime.timedelta(seconds=16293, microseconds=46490)

In [16]:
# Make predictions and calculate metrics
start = datetime.now()
train_pred = rf_plus_model.predict(X_train)
test_pred = rf_plus_model.predict(X_test)
train_rmse = mean_squared_error(y_train, train_pred, squared=False)
test_rmse = mean_squared_error(y_test, test_pred, squared=False)
end = datetime.now()
end-start

datetime.timedelta(seconds=971, microseconds=840361)

In [17]:
# Store and print results
training_results = train_rmse
testing_results= test_rmse

In [ ]:
# train_df = pd.DataFrame({'Year': year, 'Type': 'Train', 'Y_True': y_train, 'Y_Pred': train_pred})
# test_df = pd.DataFrame({'Year': year, 'Type': 'Test', 'Y_True': y_test, 'Y_Pred': test_pred})
# year_prediction_vectors = pd.concat([year_prediction_vectors, train_df, test_df])
# print(f"Year: {year}, Training RMSE: {train_rmse}, Testing RMSE: {test_rmse}")

In [18]:
training_results

56.349150908453055

In [19]:
testing_results

67.584944329421

In [25]:
import pickle

# Assume 'model' is your trained model
with open('RFPlus_full.pkl', 'wb') as file:
    pickle.dump(rf_plus_model, file)

PicklingError: Can't pickle <function _GlmPPM.<lambda> at 0x000001A3D71E2680>: attribute lookup _GlmPPM.<lambda> on imodels.importance.ppms failed